# Automated ML

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.widgets import RunDetails

import joblib
from azureml.core.model import Model

from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice

from train import clean_data

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-gridstability-automl'

experiment=Experiment(ws, experiment_name)

In [3]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-187289
aml-quickstarts-187289
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [4]:
run = experiment.start_logging()

In [5]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
ds = pd.read_csv("data/gridstability.csv")

In [7]:
x, y = clean_data(ds)

In [8]:
ds_clean= pd.concat([x, y], axis=1)
ds_clean.to_csv("data/ds_clean.csv", index=False)

In [9]:
datastore = ws.get_default_datastore()

In [10]:
datastore.upload(src_dir='data', target_path='data')

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 5 files
Target already exists. Skipping upload for data/.amlignore
Target already exists. Skipping upload for data/.amlignore.amltmp
Target already exists. Skipping upload for data/ds_clean.csv
Target already exists. Skipping upload for data/gridstability.csv
Target already exists. Skipping upload for data/requirements.txt
Uploaded 0 files


$AZUREML_DATAREFERENCE_0428fd6d7d79460f9ec87644891eb564

In [11]:
ds = Dataset.Tabular.from_delimited_files(path=[(datastore, "data/ds_clean.csv")])

In [12]:
ds.take(5).to_pandas_dataframe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,1
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,0
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,1
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,1
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
project_folder = 'gridstability-runs'

In [14]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="stabf",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [15]:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on my-cluster with default configuration
Running on remote compute: my-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-gridstability-automl,AutoML_bee27e63-7a86-4f3e-84f0-9b17aaa4a0f6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|3                             |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbal

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [19]:
remote_run.get_output()

ConfigException: ConfigException:
	Message: The model you attempted to retrieve requires 'xgboost' to be installed at '==1.3.3'. You have 'xgboost==1.3.3', please reinstall 'xgboost==1.3.3' (e.g. `pip install xgboost==1.3.3`) and rerun the previous command.
	InnerException: XGBoostError: [13:26:18] /mnt/xgboost/src/learner.cc:922: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /anaconda/envs/azureml_py38/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x7c) [0x7f9fa4c289ac]
  [bt] (1) /anaconda/envs/az

In [20]:
metrics_output = remote_run.get_pipeline_output('metrics_output')
num_file_downloaded = metrics_output.download('.', show_progress=True)

AttributeError: 'AutoMLRun' object has no attribute 'get_pipeline_output'

In [21]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

NameError: name 'metrics_output' is not defined

In [22]:
best_run, model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

ConfigException: ConfigException:
	Message: The model you attempted to retrieve requires 'xgboost' to be installed at '==1.3.3'. You have 'xgboost==1.3.3', please reinstall 'xgboost==1.3.3' (e.g. `pip install xgboost==1.3.3`) and rerun the previous command.
	InnerException: XGBoostError: [13:26:41] /mnt/xgboost/src/learner.cc:922: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /anaconda/envs/azureml_py38/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x7c) [0x7f9fa4c289ac]
  [bt] (1) /anaconda/envs/az

In [24]:
best_run_metrics['AUC_weighted']

NameError: name 'best_run_metrics' is not defined

In [25]:
model

NameError: name 'model' is not defined

In [26]:
joblib.dump(model, filename="outputs/best_model_automl.pkl")

NameError: name 'model' is not defined

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [27]:
remote_run.register_model(model_name = 'model_automl', model_path='outputs/model_automl.pkl')

TypeError: register_model() got an unexpected keyword argument 'model_path'

In [28]:
environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

NameError: name 'best_run' is not defined

In [ ]:
%run endpoint.py

In [ ]:
print(service.get_logs())

In [ ]:
#service.delete()